In [6]:
import re
import time
import requests
import numpy as np

from bs4 import BeautifulSoup
from collections import Counter
from selenium import webdriver

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd

from selenium.webdriver.common.by import By


In [7]:
url = 'https://www.usajobs.gov/Search/Results?jt=Data%20Analyst'

In [8]:
## launch headless Edge instance
options = webdriver.ChromeOptions()
options.add_argument('--headless')

driver = webdriver.Chrome(options = options)
driver.get(url)

## grab html from page
page = driver.page_source.encode('utf-8')
soup = BeautifulSoup(page, 'html.parser')

## find total number of jobs found
job_count_ele = soup.find(id = 'page-info')
print(job_count_ele.prettify())


<h4 class="usajobs-search-controls__results-count" id="page-info">
 Viewing 1 – 25 of 1554 jobs
</h4>



In [9]:
job_count = int(re.search(r'[0-9]+(?= jobs)', str(job_count_ele)).group())
print(job_count)

1554


In [10]:
## initiate variables for loop
df = pd.DataFrame(columns = ['job_title', 'job_url','job_summary', 'job_salary', 'job_location', 'job_quals', 'job_duties'])
words = []
counter = 0
start = time.time()
i = 0
for ii in range(10): 
    
    print("Starting")
    print(ii)
    page = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(page, 'html.parser')
    job_rows = soup.find_all('a', class_ = 'usajobs-search-result--core__title') 
    print("Number of jobs")
    print(len(job_rows))

    ## loop through rows, grabbing unique job posting url from each  
    for iii in range(len(job_rows)): # 
        
        ## grab url for unique job post
        job_link = re.search(r'(?<=href=\").*(?=\" id)', str(job_rows[iii])).group()
        print(job_link)
        j_url = 'https://www.usajobs.gov/'
        job_url = str(j_url + 
                      re.search(r'(?<=href=\").*(?=\" id)', 
                                str(job_rows[iii])).group()
                     )

        ## capture html from post page
        job_page = requests.get(job_url)
        job_soup = BeautifulSoup(job_page.content, 'html.parser')
        
        # grab job title
        job_title = job_soup.find('h1', class_ = 'usajobs-joa-banner__title')
        job_title = re.sub(r'\<.*\>', '', job_title.prettify())

        # grab job summary
        job_summary = job_soup.find('div', id = 'summary')
        job_summary = re.sub(r'\<.*\>', '', job_summary.prettify())
        
        # grab job description
        job_salary = job_soup.find('p', class_ = 'usajobs-joa-summary__salary salary-text-normal')
        job_salary = re.sub(r'\<.*\>', '', job_salary.prettify())
        
        # grab job location
        job_location = job_soup.find('div', class_ = "usajobs-joa-locations__body")
        job_location = re.sub(r'\<.*\>', '', job_location.prettify())
        
        # grab job qualifications
        job_quals = job_soup.find('div', id = "qualifications")
        job_quals = re.sub(r'\<.*\>', '', job_quals.prettify())
        
        job_duties = job_soup.find('div', id = "duties")
        job_duties = re.sub(r'\<.*\>', '', job_duties.prettify())
        
        
        ## move all variables to dataframe
        df.loc[i, 'job_title'] = job_title
        df.loc[i,'job_url'] = job_url
        df.loc[i,'job_summary'] = job_summary
        df.loc[i,'job_salary'] = job_salary
        df.loc[i,'job_location'] = job_location
        df.loc[i,'job_quals'] = job_quals
        df.loc[i,'job_duties'] = job_duties
        
        time.sleep(4)
        
        i = i + 1
        
    df2 = df
    df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9#&\.\+\-]', ' ')
    df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9#&\.\+\-]', ' ')
    df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9#&\.\+\-]', ' ')
    df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9#&\.\+\-]', ' ')
    df2.job_salary = df2.job_salary.str.replace('[^0-9]','-')
    df2.job_duties = df2.job_duties.str.replace('[^a-zA-Z0-9#&\.\+\-]',' ')
    df2.to_csv("data_analyst.csv")
        
    button = driver.find_element(By.CLASS_NAME, 'usajobs-search-pagination__next-page')
    driver.execute_script('arguments[0].click()', button)
    time.sleep(2)

        

        
        

Starting
0
Number of jobs
25
/job/705283300
/job/713522000
/job/713529100
/job/709017400
/job/710689400
/job/712701900
/job/713280000
/job/713280800
/job/711718200
/job/711718700
/job/712013300
/job/711218900
/job/712316700
/job/710921700
/job/711964100
/job/711964200
/job/711682100
/job/712612300
/job/712651300
/job/652648400
/job/707297000
/job/709580700
/job/710743700
/job/710744000
/job/711153200


/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:71: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h

Starting
1
Number of jobs
25
/job/711306400
/job/711306600
/job/711338200
/job/711343400
/job/711368100
/job/711571800
/job/711702800
/job/711703400
/job/712609000
/job/712628800
/job/712682000
/job/712691100
/job/713005400
/job/713151400
/job/713153000
/job/713175300
/job/713341800
/job/713618300
/job/712109700
/job/709213200
/job/709666100
/job/711116800
/job/713013700
/job/711708800
/job/711710100


/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:71: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h

Starting
2
Number of jobs
25
/job/711636400
/job/711636500
/job/711542800
/job/711462600
/job/711328500
/job/712188000
/job/713082900
/job/711774600
/job/713292100
/job/710542900
/job/711456300
/job/710983900
/job/712876800
/job/712053500
/job/712795900
/job/687971500
/job/688790200
/job/690711800
/job/710915100
/job/711141900
/job/711154600
/job/712498000
/job/710136500
/job/710139300
/job/710139500


...

/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:71: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h

Starting
8
Number of jobs
25
/job/711802400
/job/711858600
/job/711889200
/job/711893400
/job/711903200
/job/711936100
/job/711951900
/job/711953800
/job/711959200
/job/711964500
/job/711994000
/job/712003400
/job/712043200
/job/712067800
/job/712071500
/job/712102900
/job/712141700
/job/712153000
/job/712154100
/job/712179700
/job/712231200
/job/712232600
/job/712245700
/job/712280800
/job/712297800


/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:71: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h

Starting
9
Number of jobs
25
/job/711802400
/job/711858600
/job/711889200
/job/711893400
/job/711903200
/job/711936100
/job/711951900
/job/711953800
/job/711959200
/job/711964500
/job/711994000
/job/712003400
/job/712043200
/job/712067800
/job/712071500
/job/712102900
/job/712141700
/job/712153000
/job/712154100
/job/712179700
/job/712231200
/job/712232600
/job/712245700
/job/712280800
/job/712297800


/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:71: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h2j31n7v4rtsrdjgqyw0000gn/T/ipykernel_29246/1795372648.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
/var/folders/4k/6wbn_h

In [11]:
len(df)

250

In [ ]:
df_save = df

In [13]:
df[['job_title', 'job_salary', 'job_summary']]

,job_title,job_salary,job_summary
0,Data Analyst,------112-015-------172-075-----------,Summary This assignment is a ...
1,Program Analyst Data Analyst,------117-302-------175-192-------------------...,Summary This position is locate...
2,Program Analyst Data Analyst,------117-302-------175-192-------------------...,Summary This position is locate...
3,Program Analyst Data,------82-830-------183-500-----------,Summary Duty Station Locat...
4,Forensic Data Analyst,------100-694-------172-075-----------,Summary JOB DESCRIPTION F...
...,...,...,...
245,Policy Analyst,------112-015-------145-617-----------,Summary This position is l...
246,Policy Analyst,------112-015-------145-617-----------,Summary This position is l...
247,Budget Analyst,------61-275-------96-375-----------------0560...,Summary About the Position...
248,BUDGET ANALYST,------57-118-------89-835-----------,Summary This is a Direct Hire A...
